In [1]:
import numpy as np

from qibo.symbols import Z
from qibo import hamiltonians

from qiboml.models.encoding_decoding import *
from qiboml.models.ansatze import *

def run_model(model, x):
    for layer in model:
        x = layer.forward(x)
    return x

In [2]:
# binary data
model = [BinaryEncodingLayer(nqubits=3), ReuploadingLayer(nqubits=3, qubits=(0,2)), QuantumDecodingLayer(nqubits=3, qubits=(1,0))]
data = np.array([[0,0,0], [0,1,0], [1,1,0]])

for x in data:
    print(run_model(model, x).probabilities())


tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0.], shape=(8,), dtype=float64)
tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0.], shape=(8,), dtype=float64)
tf.Tensor([0. 0. 0. 0. 0. 0. 1. 0.], shape=(8,), dtype=float64)


In [3]:
# real data
nqubits = 4
observable = hamiltonians.SymbolicHamiltonian(np.prod([Z(q) for q in range(nqubits)]))
model = [PhaseEncodingLayer(nqubits, qubits=range(3)), ReuploadingLayer(nqubits), ExpectationLayer(nqubits, observable=observable)]
data = np.pi * np.random.randn(3**2).reshape(3, 3)

for x in data:
    print(run_model(model, x))

[Qibo 0.2.6|INFO|2024-06-03 19:20:11]: Using qibojit (numba) backend on /CPU:0


[(-0.6573554484285291,), (-2.7284573505919036,), (1.7086363171872503,)]
tf.Tensor(1.0, shape=(), dtype=float64)
[(-0.42238064222261756,), (-2.6976137447157047,), (-1.8614965732423072,)]
tf.Tensor(1.0, shape=(), dtype=float64)
[(6.099049952786705,), (3.8277137408169737,), (0.5745508124842935,)]
tf.Tensor(1.0, shape=(), dtype=float64)
